In [1]:
from test_fair_clustering import main
import argparse
import os
import csv
import numpy as np


Bad key savefig.frameon in file /home/anna/anaconda3/envs/fact_env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /home/anna/anaconda3/envs/fact_env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /home/anna/anaconda3/envs/fact_env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.s

In [2]:
OUTPUT_FOLDER = "outputs"
CSV_NAME = "results.csv"

lambdas = {
    "Synthetic": {
        "kmedian": 10,
        "kmeans": 10,
        # "ncut": 10,
    }, 
    "Synthetic-unequal": {
        "kmedian": 10,
        "kmeans": 10,
        # "ncut": 10,
    }, 
    "Adult": {
        "kmedian": 9000,
        "kmeans": 9000,
        # "ncut": 10,
    }, 
    "Bank": {
        "kmedian": 9000,
        "kmeans": 6000,
        # "ncut": 40,
    }, 
    "CensusII": {
        "kmedian": 500000,
        "kmeans": 500000,
        # "ncut": 100,
    }
}

n_runs = {
    "Synthetic": 30, 
    "Synthetic-unequal": 30, 
    "Adult": 20,
    "Bank": 20,
    "CensusII": 3,
}

In [3]:
def get_args(seed=1, dataset="Synthetic-unequal", cluster_option="ncut", lmbda=10):
    args = argparse.Namespace()
    
    args.plot_option_clusters_vs_lambda = True
    args.plot_option_fairness_vs_clusterE = False
    args.plot_option_balance_vs_clusterE = False
    args.plot_option_convergence = False
    args.lmbda_tune = False

    args.seed = seed
    args.dataset = dataset
    args.cluster_option = cluster_option
    args.lmbda = lmbda    

    working_dir = os.getcwd()
    args.data_dir = os.path.join(working_dir, "data")
    args.output_path = os.path.join(working_dir, OUTPUT_FOLDER)
    return args

def make_csv(dir_path, csv_path, fieldnames):
    os.makedirs(dir_path, exist_ok=True)
    if os.path.isfile(csv_path):
        with open(csv_path, "r") as f:
            reader = csv.reader(f)
            if len([row for row in reader]) > 0:
                return

    with open(csv_path, "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames)
        writer.writeheader()

def run_main(args, csv_name=CSV_NAME):
    results = main(args, logging=False, seedable=True)

    save_dict = {
        "dataset": args.dataset,
        "N": results['N'],
        "J": results['J'],
        "lmbda": args.lmbda,
        "Objective": results["clustering energy (Objective)"],
        "fairness error": results["fairness error"],
        "balance": results["balance"],
        "cluster_option": args.cluster_option,
        "time": results["time"],
        "seed": args.seed,
        "lmbda_tune": args.lmbda_tune,
        "K": results['K'],        
    }

    csv_path = os.path.join(args.output_path, csv_name)
    fieldnames = save_dict.keys()
    make_csv(args.output_path, csv_path, fieldnames)
    with open(csv_path, "a", newline='') as f:
        writer = csv.DictWriter(f, fieldnames)
        writer.writerow(save_dict)
    

In [5]:
def compare_entry(args, row):
    for key in ["dataset", "lmbda", "cluster_option", "lmbda_tune"]:
        if str(getattr(args, key)) != row[key]:
            return False
    return True

def find_same_options(csv_name, args):
    entries = []
    csv_path = os.path.join(args.output_path, csv_name)
    with open(csv_path, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if compare_entry(args, row):
                entries.append(row)
    return entries

In [6]:
for dataset in lambdas:
    for cluster_option in lambdas[dataset]:
        lmbda = lambdas[dataset][cluster_option]

        args = get_args(dataset=dataset, cluster_option=cluster_option, lmbda=lmbda)
        existing_entries = find_same_options(CSV_NAME, args)
        n = n_runs[dataset] - len(existing_entries)

        if n < 1:
            print("enough results for these settings")
            continue

        seeds = [int(entry["seed"]) for entry in existing_entries]
        seeds.append(0)
        next_seed = max(seeds) + 1
        for seed in range(next_seed, next_seed + n):
            args.seed = seed
            print()
            run_main(args, CSV_NAME)

enough results for these settings
enough results for these settings
enough results for these settings
enough results for these settings
enough results for these settings
enough results for these settings
enough results for these settings
enough results for these settings
enough results for these settings
enough results for these settings


In [7]:
# Fetch results
for dataset in lambdas:
    print(f"\n\n{dataset}")
    for cluster_option in lambdas[dataset]:
        print("\n"+cluster_option.upper())
        lmbda = lambdas[dataset][cluster_option]

        args = get_args(dataset=dataset, cluster_option=cluster_option, lmbda=lmbda)
        existing_entries = find_same_options(CSV_NAME, args)
        
        if len(existing_entries) < 1:
            print("no data yet")
            continue

        entry = existing_entries[0]
        # name = f"{dataset} (N = {entry['N']}, J = {entry['J']}, lmbda = {lmbda})"

        keys = ["Objective", "fairness error", "balance"]
        for key in keys:
            data = [float(entry[key]) for entry in existing_entries]
            mean = np.mean(data)
            std = np.std(data)

            print(f"{key}{' '*(20-len(key))} M = {mean:.2f}     SD = {std:.2f}")



Synthetic

KMEDIAN
Objective            M = 289.08     SD = 2.03
fairness error       M = 0.82     SD = 1.05
balance              M = 0.34     SD = 0.21

KMEANS
Objective            M = 203.66     SD = 2.55
fairness error       M = 2.43     SD = 1.47
balance              M = 0.27     SD = 0.44


Synthetic-unequal

KMEDIAN
Objective            M = 174.82     SD = 0.00
fairness error       M = 0.00     SD = 0.00
balance              M = 0.33     SD = 0.00

KMEANS
Objective            M = 169.15     SD = 28.20
fairness error       M = 0.25     SD = 0.74
balance              M = 0.30     SD = 0.10


Adult

KMEDIAN
Objective            M = 17763.41     SD = 244.70
fairness error       M = 0.01     SD = 0.00
balance              M = 0.42     SD = 0.01

KMEANS
Objective            M = 10175.81     SD = 165.40
fairness error       M = 0.01     SD = 0.00
balance              M = 0.40     SD = 0.01


Bank

KMEDIAN
Objective            M = 20087.08     SD = 264.69
fairness error       M = 0.04 

In [ ]:
# run_main(get_args())